In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

import pandas as pd
import cv2
import os
import math
import scipy as sp
import PIL

from glob import glob
from tqdm import tqdm

from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB6, ResNet50V2

## label별 이미지 분류

In [ ]:
# import pandas as pd
# train_df = pd.read_csv("train_df.csv")
# print(len(train_df["label"].unique()))  # type = numpy.array
# label_list = train_df["label"].unique().tolist()
# len(label_list)  # type = list

In [ ]:
# import os

# def createFolder(directory):
#     try:
#         if not os.path.exists(directory):
#             os.makedirs(directory)
#     except OSError:
#         print ('Error: Creating directory. ' +  directory)
 
# for i in range(len(label_list)):  # 레이블 개수 만큼 
#     createFolder(f'./train/{label_list[i]}')  # 레이블 폴더를 생성 

In [ ]:
# train_df.head(2)

In [ ]:
# train_folder = os.listdir('./train/')
# len(train_folder)  # -폴더개수 88 , 사진개수 = 4277개

In [ ]:
# import shutil

# for i in range(len(train_folder) - 88):  # 폴더 생성한것 88개 뺴주는 겁니다.
    
#     if train_folder[i][-3:] == "png":   # 확장자가 png면 
#         label = train_df.loc[train_df["file_name"] == f"{train_folder[i]}"]["label"][i]  # train_df에서 이미지 이름에 맞는 label을 불러와 저장
#         file_source = f'./train/{train_folder[i]}'  # train 폴더에 있는 해당 이미지를
#         file_destination = f'./train/{label}/'  # 해당 label 폴더로 이동 
#         shutil.move(file_source, file_destination)  # 이동 실행

## class별 이미지 분류

In [ ]:
# train_df = pd.read_csv("train_df.csv")
# def createFolder(directory):
#     try:
#         if not os.path.exists(directory):
#             os.makedirs(directory)
#     except OSError:
#         print ('Error: Creating directory. ' +  directory)
# class_list = train_df["class"].unique().tolist()
# for i in range(len(class_list)):
#     createFolder(f'./train_status/{class_list[i]}')

In [ ]:
# train_class_folder = os.listdir('./train_class/')
# len(train_class_folder)  # -폴더개수 15 , 사진개수 = 4277개
# for i in range(len(train_class_folder) - 15):  # 폴더 생성한것 15개 뺴주는 겁니다.
    
#     if train_class_folder[i][-3:] == "png":   # 확장자가 png면 
#         class_name = train_df.loc[train_df["file_name"] == f"{train_class_folder[i]}"]["class"][i]  # train_df에서 이미지 이름에 맞는 class_name을 불러와 저장
#         file_source = f'./train_class/{train_class_folder[i]}'  # train_class 폴더에 있는 해당 이미지를
#         file_destination = f'./train_class/{class_name}/'  # 해당 class_name 폴더로 이동 
#         shutil.move(file_source, file_destination)  # 이동 실행

# 클래스 분류 모델

In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pwd

'/content'

In [ ]:
CurrentDirectory = "./"

train_class_directory = './drive/MyDrive/efficientnet/김원빈/train_state/'

test_directory  = CurrentDirectory + '/drive/MyDrive/efficientnet/김원빈/test/'
model_class_directory = './drive/MyDrive/efficientnet/김원빈/model_cb'
tensorboard_directory = CurrentDirectory + 'Tensorboard'

In [ ]:
# batch_size: 한번에 forward & Backword 하는 샘플의 수
batch_size = 64

# Training 수
epochs = 10

# Weight 조절 parameter
LearningRate = 1e-3 # 0.001
Decay = 1e-6

img_width = 224
img_height = 224

In [ ]:
# Online-augmentation 적용 Generator
# 1. 이미지를 전부다 불러서 램 (메모리)에 올릴 수 없기 때문
# 2. 이미지는 Augmentation을 해주는게 좋아서

DATAGEN_TRAIN = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last",
    validation_split=0.10) # Train / Validation

# Online-augmentation 비적용 Generator (Test용)
DATAGEN_TEST = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last")

In [ ]:
# Generator의 instance 생성 (Train)
TRAIN_GENERATOR = DATAGEN_TRAIN.flow_from_directory(
    train_class_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode= "categorical",
    subset = "training")

VALID_GENERATOR = DATAGEN_TRAIN.flow_from_directory(
    train_class_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode="categorical",
    subset = "validation")

# Generator의 instance 생성 (Test)
TEST_GENERATOR = DATAGEN_TEST.flow_from_directory(
    test_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    shuffle = False,
    class_mode='categorical')

Found 3889 images belonging to 88 classes.
Found 388 images belonging to 88 classes.
Found 2154 images belonging to 1 classes.


In [ ]:
Model_class = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=None,
    weights='imagenet',
    input_tensor=None,
    input_shape=(img_width,img_height,3),
    pooling=None)

x = GlobalAveragePooling2D()(Model_class.output)
predictions = Dense(88, activation='softmax')(x)

Efnet_class = Model(inputs=Model_class.input, outputs=predictions)

# learning parameter를 더하여 최종 model compile
Efnet_class.compile(optimizer=
         SGD(learning_rate=LearningRate, decay=Decay, momentum=0.9, nesterov=True), 
         loss='categorical_crossentropy',
         metrics=['acc']) 

In [ ]:
# Call-back 함수

# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath=model_class_directory+'Efnet_class-{epoch:03d}-{val_loss:.4f}-{val_acc:.4f}.hdf5',
            monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# 학습과정 진행사항 확인
TB = TensorBoard(log_dir=tensorboard_directory, write_graph=True, write_images=True)

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='val_loss',factor=0.8,patience=3, verbose=1, min_lr=1e-7)

CALLBACK = [CP, TB, LR]

In [ ]:
Efnet_class.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_6 (Rescaling)        (None, 224, 224, 3)  0           ['input_7[0][0]']                
                                                                                                  
 normalization_6 (Normalization  (None, 224, 224, 3)  7          ['rescaling_6[0][0]']            
 )                                                                                                
                                                                                            

In [ ]:
########## Training Start
Efnet_class.fit_generator(
        TRAIN_GENERATOR,
        # 데이터가 너무 클 경우 1-epoch을 못하는 경우
        # steps_per_epoch=TRAIN_GENERATOR.n / batch_size,
        
        epochs=5,
        callbacks=CALLBACK,
        shuffle=True, # Training에 패턴이 존재하면 overfit이 잘 되기 때문에, Shuffle 사용해야함. 단 test에는 절대 X
        validation_data=VALID_GENERATOR)
###########

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/5
61/61 [==============================] - ETA: 0s - loss: 3.5241 - acc: 0.4528
Epoch 1: val_acc improved from -inf to 0.00000, saving model to ./drive/MyDrive/efficientnet/김원빈/model_cbEfnet_class-001-4.6374-0.0000.hdf5
61/61 [==============================] - 628s 10s/step - loss: 3.5241 - acc: 0.4528 - val_loss: 4.6374 - val_acc: 0.0000e+00 - lr: 0.0010
Epoch 2/5
61/61 [==============================] - ETA: 0s - loss: 1.6206 - acc: 0.8244
Epoch 2: val_acc improved from 0.00000 to 0.00258, saving model to ./drive/MyDrive/efficientnet/김원빈/model_cbEfnet_class-002-4.6112-0.0026.hdf5
61/61 [==============================] - 219s 4s/step - loss: 1.6206 - acc: 0.8244 - val_loss: 4.6112 - val_acc: 0.0026 - lr: 0.0010
Epoch 3/5
61/61 [==============================] - ETA: 0s - loss: 1.1714 - acc: 0.8349
Epoch 3: val_acc did not improve from 0.00258
61/61 [==============================] - 213s 3s/step - loss: 1.1714 - acc: 0.8349 - val_loss: 4.6733 - val_acc: 0.0026 - lr

# 불러오기

In [ ]:
Efnet_class.load_weights("./클래스분류모델CP/Efnet_class-004-1.8732-0.4360.hdf5")

In [ ]:
# 예측용 제너레이터는 폴더 1개에 박아두고 만들어도 됨
TEST_GENERATOR = DATAGEN_TEST.flow_from_directory(
    test_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    shuffle = False,
    class_mode='categorical')

Found 2154 images belonging to 1 classes.


In [ ]:
pred_train = Efnet_class.predict_generator(TRAIN_GENERATOR, verbose=1)

C:\Users\user\AppData\Local\Temp/ipykernel_14516/4134788626.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred_train = Efnet_class.predict_generator(TRAIN_GENERATOR, verbose=1)


  5/121 [>.............................] - ETA: 2:32

KeyboardInterrupt: 

In [ ]:
pred_train[0]

In [ ]:
# train_state 폴더 넣었을 때 1번값
pred_train[0]

In [ ]:
# test 예측
pred = Efnet_class.predict_generator(TEST_GENERATOR, verbose=1)

In [ ]:
pred[6]